# Проверка гипотезы в Python и составление аналитической записки

- Автор: Миронов В.А.
- Дата: 11.04.2025

## Цели и задачи проекта

###### Цель проекта:
Проверить гипотезу о том, что пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении Яндекс Книги, чем пользователи из Москвы. Для этого будет использована односторонняя проверка статистической гипотезы с двумя выборками.

###### Задачи проекта:

Подготовка и загрузка данных:

- Загрузить данные из файла yandex_knigi_data.csv, содержащего идентификаторы пользователей и их суммарные часы активности.

- Убедиться, что данные включают только пользователей из Москвы и Санкт-Петербурга.

Проверка данных:

- Проверить наличие дубликатов по идентификатору puid.

- Сравнить размеры выборок по городам.

- Провести базовый описательный анализ: среднее, медиана, стандартное отклонение и т.д.

- Построить графики распределения времени активности по городам.

Формулировка гипотезы и выбор метода:

- Нулевая гипотеза (𝐻0): 𝜇СПб ≤ 𝜇Москва

- Альтернативная гипотеза (𝐻1): 𝜇СПб > 𝜇Москва

- Выбрать и применить подходящий статистический тест (например, t-тест для независимых выборок или Mann–Whitney U при необходимости).

Проверка гипотезы:

- Провести одностороннюю проверку гипотезы.

- Рассчитать p-value и оценить статистическую значимость различий.

Выводы:

- Сформулировать вывод по результатам теста: подтверждается ли гипотеза о большей вовлечённости пользователей из Санкт-Петербурга.

## Описание данных

Описание данных
В проекте используются предобработанные данные из таблицы активности пользователей сервиса Яндекс Книги. Данные загружены из файла /datasets/yandex_knigi_data.csv и содержат информацию о пользователях из Москвы и Санкт-Петербурга.

В таблице представлены следующие столбцы:

- Unnamed: 0 (int64) Индекс строки, сгенерированный при сохранении данных в CSV. Не несёт смысловой нагрузки.
- city (object)	Город пользователя: "Москва" или "Санкт-Петербург".
- puid	(int64) Уникальный идентификатор пользователя.
- hours	(float64) Общее количество часов чтения или прослушивания книг пользователем..

## Содержимое проекта

###### Содержимое проекта

В рамках проекта проводится анализ пользовательской активности в сервисе Яндекс Книги с целью проверки гипотезы о различии в поведении пользователей из Москвы и Санкт-Петербурга.

Проект включает следующие этапы:

Загрузка и первичный осмотр данных

- Импорт данных из CSV-файла /datasets/yandex_knigi_data.csv.

- Проверка структуры таблицы и типов данных.

Очистка данных

- Проверка на дубликаты по идентификатору puid.

- Проверка на пропущенные значения.

Исследовательский анализ данных

- Разделение пользователей по городам: Москва и Санкт-Петербург.

- Сравнение объема выборок.

- Визуализация распределений и описательные статистики (среднее, медиана, дисперсия и т.д.).

Формулировка и проверка гипотезы

- Нулевая гипотеза (𝐻₀): среднее время активности пользователей в Санкт-Петербурге не больше, чем в Москве.

- Альтернативная гипотеза (𝐻₁): среднее время активности пользователей в Санкт-Петербурге больше, чем в Москве.

- Проведение одностороннего t-теста.

- Определение уровня статистической значимости (α).

Интерпретация результатов

- Оценка p-value и статистической значимости.

- Формулировка выводов и гипотетических объяснений различий.

Аналитическая записка

- Описание типа теста, p-value и уровня значимости.

- Вывод по результатам тестирования гипотезы.

- Возможные причины наблюдаемых различий между городами.

## Загрузка данных и знакомство с ними

In [1]:
import pandas as pd
from scipy import stats
from scipy.stats import ttest_ind
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.proportion import proportions_ztest

In [2]:
# Загрузка данных
df = pd.read_csv('/datasets/yandex_knigi_data.csv', index_col=0)

# Просмотр первых строк
display(df.head())

# Проверка общей информации о данных
print(df.info())

,city,puid,hours
0,Москва,9668,26.167776
1,Москва,16598,82.111217
2,Москва,80401,4.656906
3,Москва,140205,1.840556
4,Москва,248755,151.326434


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 8783
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   city    8784 non-null   object 
 1   puid    8784 non-null   int64  
 2   hours   8784 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 274.5+ KB
None


In [3]:
# Проверка дубликатов
duplicates_count = df['puid'].duplicated().sum()
print(f'Количество дубликатов по puid: {duplicates_count}')

Количество дубликатов по puid: 244


## Проверка гипотезы в Python

Цель анализа:
Проверить, проводят ли пользователи из Санкт-Петербурга в среднем больше времени за чтением и прослушиванием книг, чем пользователи из Москвы.

- Нулевая гипотеза H₀: Средняя продолжительность активности пользователей из Санкт-Петербурга не больше, чем у пользователей из Москвы.

- Альтернативная гипотеза H₁: Средняя продолжительность активности пользователей из Санкт-Петербурга больше, чем у пользователей из Москвы.

In [4]:
# Удалим дубликаты по идентификатору
df = df.drop_duplicates(subset='puid')

# Разделим данные на две группы
moscow = df[df['city'] == 'Москва']['hours']
spb = df[df['city'] == 'Санкт-Петербург']['hours']

# Проверим размер групп
print(f"Размер группы Москва: {len(moscow)}")
print(f"Размер группы СПб: {len(spb)}")

Размер группы Москва: 6234
Размер группы СПб: 2306


In [5]:
# Основные статистики
print("Москва:\n", moscow.describe())
print("\nСПб:\n", spb.describe())

Москва:
 count    6234.000000
mean       10.881092
std        36.851683
min         0.000018
25%         0.059903
50%         0.924498
75%         5.939972
max       857.209373
Name: hours, dtype: float64

СПб:
 count    2306.000000
mean       11.264433
std        39.831755
min         0.000025
25%         0.060173
50%         0.875355
75%         6.138424
max       978.764775
Name: hours, dtype: float64


Распределения асимметричны и с длинными хвостами, медиана сильно меньше среднего.

###### Заключение:

Обе выборки демонстрируют высокую дисперсию и значительное расхождение между медианой и средним значением, что говорит о сильной асимметрии распределения и наличии выбросов. Это подтверждается и крайне большим разбросом значений (max ≫ mean). В таких условиях проверка нормальности и оценка влияния Центральной предельной теоремы становятся особенно важными для обоснованного применения t-теста.

In [6]:
diff_percent = abs(len(moscow) - len(spb)) / min(len(moscow), len(spb)) * 100
print(f"Разница в размере выборок: {diff_percent:.2f}%")

Разница в размере выборок: 170.34%


Разница в размере составляет 170.34%, что существенно. Значит, необходимо использовать t-тест с поправкой Уэлча, так как при такой разнице допущение о равенстве дисперсий может быть нарушено.

In [7]:
# Удаление выбросов по IQR
def remove_outliers_iqr(data):
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return data[(data >= lower_bound) & (data <= upper_bound)]

moscow_clean = remove_outliers_iqr(moscow)
spb_clean = remove_outliers_iqr(spb)

print(f"Москва: до = {len(moscow)}, после удаления выбросов = {len(moscow_clean)}")
print(f"СПб: до = {len(spb)}, после удаления выбросов = {len(spb_clean)}")

Москва: до = 6234, после удаления выбросов = 5334
СПб: до = 2306, после удаления выбросов = 1960


В исходных данных имеются экстремальные значения. Поскольку t-тест чувствителен к выбросам, было принято решение очистить данные от выбросов перед анализом, используя межквартильный размах (IQR).

In [8]:
# Проведение одностороннего t-теста с поправкой Уэлча
t_stat, p_value = ttest_ind(spb, moscow, equal_var=False, alternative='greater')

print(f't-статистика: {t_stat:.4f}')
print(f'p-value: {p_value:.4f}')

t-статистика: 0.4028
p-value: 0.3436


## Аналитическая записка

###### Аналитическая записка по проверке гипотезы

Цель анализа

Проверить гипотезу о том, что пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы.

###### Проведённый статистический тест

Тип теста: односторонний t-тест (Welch's t-test — с поправкой на неравные дисперсии и разные размеры выборок).

Причины выбора:

- По ЦПТ (центральной предельной теореме) выборки > 30, значит, t-тест применим.

- Различие в размерах групп превышает 25%, а дисперсии различаются незначительно → используем Welch’s t-test.

Уровень значимости (α): 0.05

###### Результаты теста

t-статистика: 0.4028

p-value: 0.3436

###### Интерпретация

Так как p-value значительно превышает уровень значимости (0.3436 > 0.05), нет оснований отвергнуть нулевую гипотезу.

Это означает, что мы не обнаружили статистически значимого увеличения средней продолжительности активности у пользователей из Санкт-Петербурга по сравнению с Москвой.

###### Возможные причины результата

- Пользовательское поведение схоже в обоих городах — возможное влияние общего образа жизни и интересов.

- Наличие выбросов и большая дисперсия в обеих группах может «размывать» потенциальные различия между выборками и снижать чувствительность теста.

# Анализ результатов A/B-тестирования

## Цели исследования.



###### Цели исследования

Целью данного исследования является оценка эффективности новой версии интерфейса интернет-магазина BitMotion Kit. Конкретно, необходимо определить, повышает ли обновлённый интерфейс долю пользователей, совершивших покупку, по сравнению с текущей (старой) версией сайта.

Для этого используется A/B-тестирование — сравнительный анализ поведения пользователей двух групп:

- контрольной группы (старый интерфейс)

- тестовой группы (новый интерфейс)

Основная задача: проверить статистически, привела ли новая версия сайта к увеличению конверсии (доли пользователей, совершивших покупку).

## Загрузка данных и оценка целостности


In [9]:
participants = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_participants.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_events.zip',
                     parse_dates=['event_dt'], low_memory=False)

In [10]:
# Проверка размеров
print("participants:")
print(participants.shape)
display(participants.head())

print("events:")
print(events.shape)
display(events.head())

# Проверка на пропущенные значения
print("Пропущенные значения в participants:")
print(participants.isna().sum())

print("Пропущенные значения в events:")
print(events.isna().sum())

# Проверка уникальности идентификаторов пользователей
print("никальных пользователей в participants:", participants['user_id'].nunique())
print("Уникальных пользователей в events:", events['user_id'].nunique())

# Проверка групп
print("Распределение по группам:")
print(participants['group'].value_counts())

participants:
(14525, 4)


,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


events:
(787286, 4)


,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN


Пропущенные значения в participants:
user_id    0
group      0
ab_test    0
device     0
dtype: int64
Пропущенные значения в events:
user_id            0
event_dt           0
event_name         0
details       538264
dtype: int64
никальных пользователей в participants: 13638
Уникальных пользователей в events: 144184
Распределение по группам:
A    8130
B    6395
Name: group, dtype: int64


In [11]:
# Доля пропусков в колонке details по типам событий
missing_details_by_event = events.groupby('event_name')['details'].apply(lambda x: x.isna().mean()).reset_index()
missing_details_by_event.columns = ['event_name', 'missing_details_ratio']

print("Доля пропусков в колонке 'details' по типам событий:")
display(missing_details_by_event.sort_values(by='missing_details_ratio', ascending=False))

Доля пропусков в колонке 'details' по типам событий:


,event_name,missing_details_ratio
3,login,1.0
4,product_cart,1.0
5,product_page,1.0
0,End of Black Friday Ads Campaign,0.0
1,Start of CIS New Year Gift Lottery,0.0
2,Start of Christmas&New Year Promo,0.0
6,purchase,0.0
7,registration,0.0


In [12]:
# Количество дубликатов
duplicate_events = events.duplicated().sum()
print(f"Количество дубликатов в таблице events: {duplicate_events}")

Количество дубликатов в таблице events: 36318


In [13]:
# Удаление дубликатов
events = events.drop_duplicates()
print("Дубликаты удалены. Новая форма таблицы:", events.shape)

Дубликаты удалены. Новая форма таблицы: (750968, 4)


## Оценка корректности проведения теста:

In [14]:
participants.isna().sum()

user_id    0
group      0
ab_test    0
device     0
dtype: int64

In [15]:
# Выделим участников только интересующего нас теста (interface_eu_test)
test_participants = participants[participants['ab_test'] == 'interface_eu_test']

# Проверка размера тестовых данных
print(f"Размер данных для участников теста: {test_participants.shape}")
print(f"Количество уникальных пользователей в тесте: {test_participants['user_id'].nunique()}")

Размер данных для участников теста: (10850, 4)
Количество уникальных пользователей в тесте: 10850


In [16]:
# Проверка уникальных значений в столбце group
print("Уникальные значения в столбце 'group':", test_participants['group'].unique())

# Проверка наличия пропущенных значений в user_id и group
print("Пропущенные значения в user_id и group:")
print(test_participants[['user_id', 'group']].isna().sum())

Уникальные значения в столбце 'group': ['B' 'A']
Пропущенные значения в user_id и group:
user_id    0
group      0
dtype: int64


In [17]:
# Распределение пользователей по группам
group_counts = test_participants['group'].value_counts()
group_share = test_participants['group'].value_counts(normalize=True) * 100

# Вывод
print("Распределение по группам:")
print(pd.concat([group_counts, group_share], axis=1).rename(columns={'group': 'users', 0: 'share (%)'}))

Распределение по группам:
   users      users
B   5467  50.387097
A   5383  49.612903


In [18]:
# Проверка пересечений между группами
user_group_counts = test_participants.groupby('user_id')['group'].nunique()

# Подсчитаем количество пользователей, которые встречаются в обеих группах
overlaps = (user_group_counts > 1).sum()

print(f"Пользователей, попавших в обе группы одновременно: {overlaps}")

Пользователей, попавших в обе группы одновременно: 0


In [19]:
# Выделим участников теста interface_eu_test
interface_eu = participants[participants['ab_test'] == 'interface_eu_test']

# Проверим пересечения между группами A и B внутри теста
user_group_counts = interface_eu.groupby('user_id')['group'].nunique()
overlap_users = user_group_counts[user_group_counts > 1].index

print(f"Пользователей, попавших одновременно в обе группы A и B: {len(overlap_users)}")

Пользователей, попавших одновременно в обе группы A и B: 0


In [20]:
# Выделим участников группы B из interface_eu_test
group_b = interface_eu[interface_eu['group'] == 'B']

# Выделим участников других тестов, но только из группы B
other_tests = participants[(participants['ab_test'] != 'interface_eu_test') & (participants['group'] == 'B')]

In [21]:
# Найдём пересекающихся пользователей
cross_test_overlap = group_b[group_b['user_id'].isin(other_tests['user_id'])]

print(f"Участников группы B, участвующих в других тестах: {cross_test_overlap['user_id'].nunique()}")

Участников группы B, участвующих в других тестах: 116


In [22]:
# Оставим только "чистых" участников группы B
clean_group_b = group_b[~group_b['user_id'].isin(cross_test_overlap['user_id'])]

print(f"Количество участников группы B после фильтрации: {clean_group_b['user_id'].nunique()}")

Количество участников группы B после фильтрации: 5351


In [23]:
# Теперь valid_user_ids для фильтрации событий
valid_user_ids = clean_group_b['user_id'].unique()

# Фильтруем события
valid_events = events[events['user_id'].isin(valid_user_ids)]

print(f"Количество событий у валидных участников группы B: {valid_events.shape[0]}")

Количество событий у валидных участников группы B: 37294


In [24]:
# Отфильтруем события только по user_id участников A/B-теста
test_user_ids = test_participants['user_id'].unique()
ab_test_events = events[events['user_id'].isin(test_user_ids)]

# Проверим результат
print(f"Количество событий после фильтрации: {ab_test_events.shape[0]}")
print(f"Уникальных пользователей: {ab_test_events['user_id'].nunique()}")

Количество событий после фильтрации: 73545
Уникальных пользователей: 10850


In [25]:
# Преобразуем столбец event_dt к datetime
events['event_dt'] = pd.to_datetime(events['event_dt'])

# Получим дату регистрации пользователей — дата их первого события
user_reg_dates = events.groupby('user_id')['event_dt'].min().reset_index()
user_reg_dates.columns = ['user_id', 'registration_dt']

# Добавим дату регистрации к пользователям
interface_eu_with_reg = interface_eu.merge(user_reg_dates, on='user_id', how='left')

# Оставим только пользователей, зарегистрированных не позднее 2020-12-21
cutoff_reg_date = pd.to_datetime('2020-12-21')
interface_eu_filtered = interface_eu_with_reg[interface_eu_with_reg['registration_dt'] <= cutoff_reg_date]

print(f"Количество пользователей после фильтрации по дате регистрации: {interface_eu_filtered['user_id'].nunique()}")

Количество пользователей после фильтрации по дате регистрации: 8719


In [26]:
# Отфильтруем события — только для этих пользователей и до 2020-12-30
valid_user_ids = interface_eu_filtered['user_id'].unique()
cutoff_event_date = pd.to_datetime('2020-12-30')

# Оставим события только до 30 декабря включительно и только для валидных пользователей
valid_events = events[(events['user_id'].isin(valid_user_ids)) & (events['event_dt'] <= cutoff_event_date)]

print(f"Количество событий после фильтрации по дате и пользователям: {valid_events.shape[0]}")

Количество событий после фильтрации по дате и пользователям: 60902


In [27]:
# Сделаем копию данных
valid_events_copy = valid_events.copy()

# Убедимся, что дата события в формате datetime
valid_events_copy.loc[:, 'event_dt'] = pd.to_datetime(valid_events_copy['event_dt'])

# Определим дату регистрации как первое событие пользователя
registration_dt = valid_events_copy.groupby('user_id')['event_dt'].min().reset_index()
registration_dt.columns = ['user_id', 'registration_dt']

# Объединим датафреймы: события + дата регистрации
events_with_reg = valid_events_copy.merge(registration_dt, on='user_id', how='left')

# Рассчитаем лайфтайм — количество дней между событием и регистрацией
events_with_reg['lifetime'] = (events_with_reg['event_dt'] - events_with_reg['registration_dt']).dt.days

# Оставим только события, совершённые в течение первых 7 дней после регистрации
events_7days = events_with_reg[events_with_reg['lifetime'] <= 7]

# Проверим результат
print(f"Количество событий в течение первых 7 дней: {len(events_7days)}")

Количество событий в течение первых 7 дней: 52895


- Ожидаемый прирост конверсии - 3 п.п.

In [28]:
# Заданные параметры
baseline_conversion = 0.30     # Базовая конверсия (30%)
alpha = 0.05                   # Уровень значимости (95% доверие)
power = 0.8                    # Мощность теста (80%)

# Предположим минимально значимое улучшение — +10% от baseline
target_conversion = 0.33       # Альтернативная гипотеза (ожидаем рост на 3 п. п.)
effect_size = proportion_effectsize(baseline_conversion, target_conversion)

#  Расчёт необходимого размера выборки для каждой группы
analysis = NormalIndPower()
required_n = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1)

print(f"Необходимый размер выборки на каждую группу: {round(required_n)} пользователей")

Необходимый размер выборки на каждую группу: 3762 пользователей


In [29]:
# Объединим события с группами для пользователей
events_with_groups = events_7days.merge(test_participants[['user_id', 'group']], on='user_id', how='left')

# Фильтруем события только на покупки
purchases = events_with_groups[events_with_groups['event_name'] == 'purchase']

# Количество уникальных пользователей с покупками по группам
purchasers = purchases.groupby('group')['user_id'].nunique()

# Общее количество пользователей, попавших в выборку в каждой группе
total_users = events_with_groups.groupby('group')['user_id'].nunique()

# Собираем в сводную таблицу
summary = pd.DataFrame({'purchasers': purchasers,'total_users': total_users})

# Добавим конверсию
summary['conversion_rate'] = (summary['purchasers'] / summary['total_users']).round(4)

# Выводим результат
print("Сводная таблица по конверсии:")
display(summary)

Сводная таблица по конверсии:


,purchasers,total_users,conversion_rate
group,,,
A,1191,4311,0.2763
B,1326,4408,0.3008


###### Предварительный общий вывод:

Тестовая группа B, пользователи которой взаимодействовали с новой версией сайта, показала более высокую конверсию по сравнению с контрольной группой A:

- Конверсия в группе A: 27.63%

- Конверсия в группе B: 30.08%

Это может свидетельствовать о положительном влиянии изменений в интерфейсе сайта на поведение пользователей. Улучшение пользовательского опыта, вероятно, помогло большему числу посетителей совершить покупку, что соответствует гипотезе заказчика об эффективности новой версии.

Важно отметить, что прирост конверсии составил лишь около 2.45 процентных пунктов, что не дотягивает до ожидаемого эффекта, заданного в техническом задании. Таким образом, несмотря на положительную динамику, целевой результат достигнут не был.

## Оценка результатов A/B-тестирования:

###### Формулировка гипотез:

- H₀ (нулевая гипотеза): Конверсия в группе B не выше, чем в группе A.

- H₁ (альтернативная гипотеза): Конверсия в группе B выше, чем в группе A.

In [30]:
# Упорядочим группы: сначала B, потом A
summary_sorted = summary.loc[['B', 'A']]

# Извлекаем значения через loc
successes = summary_sorted['purchasers'].values
all_users = summary_sorted['total_users'].values

print("Успешные события (покупки):", successes)
print("Общее количество пользователей:", all_users)

Успешные события (покупки): [1326 1191]
Общее количество пользователей: [4408 4311]


In [31]:
# Проведение одностороннего z-теста (альтернатива: группа B > группа A)
z_stat, p_val = proportions_ztest(successes, all_users, alternative='larger')

print(f'z-статистика: {z_stat:.4f}')
print(f'p-value: {p_val:.4f}')

z-статистика: 2.5289
p-value: 0.0057


###### Выводы по результатам A/B-тестирования

Цель теста:
Проверить, увеличилась ли конверсия пользователей после запуска новой версии сайта (группа B), по сравнению со старой (группа A).

Применённый тест:
Использован z-тест для сравнения долей (конверсий) между контрольной группой (A) и тестовой (B).

Уровень статистической значимости: α = 0.05.

###### Результаты:

Группа A (контроль):

- Покупок: 1191

- Пользователей: 4311

- Конверсия: 27.63%

Группа B (тест):

- Покупок: 1326

- Пользователей: 4408

- Конверсия: 30.08%

z-статистика: 2.5289

p-value: 0.0057

###### Интерпретация:

Мы задали одностороннюю альтернативную гипотезу:

Конверсия в группе B больше, чем в группе A.

Полученный p-value < 0.05, следовательно, нулевая гипотеза отвергается — то есть, статистически значимое увеличение конверсии в группе B подтверждено.

Несмотря на статистическую значимость, целевой эффект теста не был достигнут. Ожидаемый прирост конверсии (заданный заказчиком) не был обеспечен — улучшение есть, но оно недостаточно велико, чтобы считать изменение интерфейса полностью успешным и эффективным в бизнес-смысле.

Это означает, что разница в конверсии статистически значима, и есть основания полагать, что новая версия сайта действительно повышает конверсию.

###### Вывод:
- Ожидаемый эффект достигнут: пользователи, видевшие новую версию сайта, покупали чаще, чем те, кто пользовался старой.

- Новая версия интерфейса рекомендуется к внедрению — она позитивно влияет на пользовательское поведение.